# GxE Variant Clustering and Transcription Factor Binding Site Analysis

This notebook contains code for analyzing clustering of variants with gene-by-environment interactions among each annotation type. In addition, it contains analysis of overlap of these clusters with previously identified transcription factor binding sites (TFBS) in yeast, as well as code to generate plots for figure 3G and figure S4



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import roman
import numpy as np
import warnings
from scipy.stats import hypergeom
import seaborn as sns
warnings.filterwarnings('ignore')

plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 7

In [ ]:
annotations = pd.read_csv('../data_tables/ergosterol_annotations.tsv',sep = '\t')
gxe = pd.read_csv('../data_tables/ergosterol_pathway_gxe_interactions.tsv', sep = '\t')
fitness = pd.read_csv('../data_tables/Fitness_files/ergosterol_pathway_fitness.tsv',sep='\t')
annotated_gxe = annotations.merge(gxe, 'inner', 'var_id')
annotated_fitness =  annotations.merge(fitness, 'inner', 'var_id')

In [ ]:
def get_GxE_vars(annotated_gxe_df, annotated_fitness_df):
    '''This code generates the list of GxE variants based on our stringent two-part definition:
    1) the variant has a GxE term with FDR<0.01, and 2) for that term, the direction
    of effect in the two conditions is in opposite directions.
    inputs:
        annotated_gxe_df (pandas DataFrame): table containing pairwise GxE coefficients and pvalues for variants,
        annotated by genomic location.
        annotated_fitness_df (pandas DataFrame):  table containing fitness values for variants in 
        all relevant conditions,annotated by genomic location.
    returns:
        gxe_vars (list of strings): list of variants which show GxE according to our definition 
        (not unique, many represented multiple times) 
    '''
    gxe_vars =[] 
    for i, row in annotated_gxe_df[(annotated_gxe_df['padj']<0.01)].iterrows():
        var = row['var_id']
        conds = row['gxe'].split('-')
        if annotated_fitness_df[(annotated_fitness_df['var_id']==var)&\
                                (annotated_fitness_df['condition'].isin(conds))]['coef'].prod()<0:
            gxe_vars.append(var)
    return gxe_vars



gxe_vars =get_GxE_vars(annotated_gxe, annotated_fitness) 

gxe_hit_variants = set(gxe_vars)
all_variants = annotated_gxe['var_id'].unique()

all_variant_annotations = annotations[(annotations['var_id'].isin(all_variants))]
hit_variant_annotations = annotations[(annotations['var_id'].isin(gxe_hit_variants))]

In [ ]:
def clustering_calculation(annotation_type_hits,distance):
    '''
    Takes in a pandas DataFrame of hit GxE variants with a given annotation and 
    calculates how many hit variants at unique positions are within clusters with 
    each other within a certain distance threshold.
    
    inputs: 
        annotation_type_hits (pandas DataFrame): DataFrame with hit variants 
        (needs chrom, SNP_chr_pos, and var_id columns)
        
        distance (int): integer distance for the cluster size
    returns:
        clustered_hits (int): number of unique GxE variant positions within clusters
    '''
    
    for i,row in annotation_type_hits.iterrows():
        chrom = row['chrom']
        pos = row['SNP_chr_pos']
        var = row['var_id']
        #for each GxE hit variant, this checks if there is another GxE hit variant
        #within the distance threshold which is not the same variant and is not 
        #at the same position. If so, the row with the variant being checked has
        # a true value for the column 'in_cluster.'
        if annotation_type_hits[(annotation_type_hits['chrom']==chrom)&\
                                (abs(annotation_type_hits['SNP_chr_pos']-pos)<distance)&\
                                (annotation_type_hits['var_id']!=var)&\
                                (annotation_type_hits['SNP_chr_pos']!=pos)]['var_id'].nunique():
            annotation_type_hits.loc[i,'in_cluster'] = True
        else:
            annotation_type_hits.loc[i,'in_cluster'] = False
            
    #gets the number of GxE variants at unique positions in the genome which are within a cluster        
    clustered_hits = annotation_type_hits.loc[annotation_type_hits['in_cluster']]['SNP_chr_pos'].nunique() 
    return clustered_hits

def calculating_clustering_pval(distance,annotation_type, num_perms = 1000, seed = 2022):
    '''
    Takes in an annotation type and 
    calculates how many hit variants at unique positions are within clusters with 
    eachother within a certain distance threshold, and a pvalue for this number using
    a permutation approach.
    
    inputs: 
    
        distance (int): integer distance for the cluster size
        annotation_type (String): name of annotation type
        num_perms (int): number of permutations to perform
        seed (int): seed for np.random
        hit_variant_annotations (pandas DataFrame):defined above, all annotations for hit variants
        all_variant_annotations (pandas DataFrame): defined above, all annotations for hit variants
    returns:
        distance threshold (int)
        p-value (float)
        number of hits in clusters (int)
    '''
    anno_type_hits = hit_variant_annotations[hit_variant_annotations['Consequence']==annotation_type]
    num_clustered_hits =  clustering_calculation(anno_type_hits,distance)




    #permuting by randomly choosing anno_type variants to be hits (the same number of hits as in the real dataset),
    #and then seeing how many of these randomly chosen variants are within a cluster as defined above

    np.random.seed(seed)

    num_anno_type_hits = hit_variant_annotations[hit_variant_annotations['Consequence']==annotation_type].var_id.nunique()
    anno_type_variants=  all_variant_annotations[(all_variant_annotations['Consequence'] == annotation_type)]
    num_permutation_clusters = []
    for j in range(num_perms):
        permutation_annos_hits = np.random.choice(anno_type_variants['var_id'], size = num_anno_type_hits, replace = False)
        permutation_annos = anno_type_variants[anno_type_variants['var_id'].isin(permutation_annos_hits)]
        num_permutation_clusters.append(clustering_calculation(permutation_annos, distance))
    num_permutation_clusters  = np.array(num_permutation_clusters )
    
    #calculating a permutation p-value by counting how many permuted datasets have as many or 
    #more clustered variants as the real data set.
    p = (len(num_permutation_clusters[num_permutation_clusters>= num_clustered_hits])/num_perms)
    
    # minimum p-value is 1/num_perms, e.g 0.001 for 1000
    if p == 0:
        p = 1/num_perms
  
    
    return(distance, p, num_clustered_hits)

## Checking for clustering of GxE Variants at different distance thresholds

### Promoter Variants

In [ ]:
#identifying "clusters"--hit variants within x bp of another hit variants, excluding hits that only 
#cluster with hits at the same genomic location, at a variety of distance thresholds
dists = []
pvalues = []
for i in np.arange(0.5,10,0.5):
    dist_thresh = round(2**i)
    if dist_thresh in dists:
        continue
    dist, p, num_clust_hits = calculating_clustering_pval(dist_thresh,'upstream_gene_variant')
    dists.append(dist)
    pvalues.append(p)
    
    
#plotting permutation p-values for clustering at each distance threshold   
fig, ax = plt.subplots(figsize = (2.5,2.5))
ax.axhline(-np.log10(0.05),color = 'red', linestyle = 'dashed', linewidth = 1)
#windows relating to yeast transcription factor binding site size, see analysis in section
#"Overlapping TFBS Data with Clusters and Promoter Hits" of notebook
#TFBS size range
ax.axvspan(4,52, color = 'grey', alpha = .2)
#mean TFBS size
ax.axvline(8.7,color = 'blue', linestyle = 'dashed', linewidth = 1)
#max distance from gene body for our designed variants
ax.axvline(500,color = 'black', linestyle = 'dashed', linewidth = 1)
plt.legend(['p = 0.05', 'ORegAnno TFBS range', 'Average TFBS length','Promoter window length'])
ax.scatter(dists, -np.log10(np.array(pvalues)), s = 5)

plt.title('Promoter variant clustering')
ax.set_xscale('log',base=2)
plt.ylabel('-log10(p)')
plt.xlabel('Cluster Size Cutoff')
plt.savefig('../GxE_Figures/Supplementary_Figures/figS4_upstream.svg')

### Downstream Variants

In [ ]:
dists = []
pvalues = []
for i in np.arange(0.5,10,0.5):
    dist_thresh = round(2**i)
    if dist_thresh in dists:
        continue
    dist, p, num_clust_hits = calculating_clustering_pval(dist_thresh,'downstream_gene_variant')
    dists.append(dist)
    pvalues.append(p)
    
fig, ax = plt.subplots(figsize = (2.5,2.5))
ax.axhline(-np.log10(0.05),color = 'red', linestyle = 'dashed', linewidth = 1)

plt.legend(['p = 0.05'])
ax.scatter(dists, -np.log10(np.array(pvalues)), s = 5)

plt.title('Downstream variant clustering')
ax.set_xscale('log',base=2)
plt.ylabel('-log10(p)')
plt.xlabel('Cluster Size Cutoff')
plt.savefig('../GxE_Figures/Supplementary_Figures/figS4_downstream.svg')

### Missense Variants

In [ ]:
#identifying "clusters"--hit variants within 8 bp of another hit variants, excluding hits that only 
#cluster with hits at the same genomic location
for i in np.arange(0.5,10,0.5):
    dist_thresh = round(2**i)
    if dist_thresh in dists:
        continue
    dist, p, num_clust_hits = calculating_clustering_pval(dist_thresh,'missense_variant')
    dists.append(dist)
    pvalues.append(p)
    
fig, ax = plt.subplots(figsize = (2.5,2.5))
ax.axhline(-np.log10(0.05),color = 'red', linestyle = 'dashed', linewidth = 1)


ax.scatter(dists, -np.log10(np.array(pvalues)), s = 5)

plt.title('Missense variant clustering')
ax.set_xscale('log',base=2)
plt.ylabel('-log10(p)')
plt.xlabel('Cluster Size Cutoff')
plt.savefig('../GxE_Figures/Supplementary_Figures/figS4_missense.svg')

### Synonymous Variants

In [ ]:
#identifying "clusters"--hit variants within 8 bp of another hit variants, excluding hits that only 
#cluster with hits at the same genomic location
for i in np.arange(0.5,10,0.5):
    dist_thresh = round(2**i)
    if dist_thresh in dists:
        continue
    dist, p, num_clust_hits = calculating_clustering_pval(dist_thresh,'synonymous_variant')
    dists.append(dist)
    pvalues.append(p)
    
fig, ax = plt.subplots(figsize = (2.5,2.5))
ax.axhline(-np.log10(0.05),color = 'red', linestyle = 'dashed')


plt.legend(['p = 0.05'])
ax.scatter(dists, -np.log10(np.array(pvalues)), s = 5)

plt.title('Synonymous variant clustering')
ax.set_xscale('log',base=2)
plt.ylabel('-log10(p)')
plt.xlabel('Cluster Size Cutoff')
plt.savefig('../GxE_Figures/Supplementary_Figures/figS4_synonymous.svg')

## Calculating final p-value for 8 base pair clusters of promoter variants

In [ ]:
#Using a distance threshold of 8 because it is roughly the average size of a yeast TFBS

dist_thresh = 8

dist, p_13, num_clust_hits = calculating_clustering_pval(dist_thresh,'upstream_gene_variant', num_perms = 5000, seed = 13)

dist, p_2022, num_clust_hits = calculating_clustering_pval(dist_thresh,'upstream_gene_variant', num_perms = 5000, seed = 2022)

In [ ]:
print('permutation seed #1 p='+ str(p_13))
print('permutation seed #2 p='+ str(p_2022))

print('Average p-value: '+str((p_13+p_2022)/2))

In [ ]:
promoter_hits = annotated_gxe[(annotated_gxe['Consequence']=='upstream_gene_variant')&(annotated_gxe['padj']<.01)]
for i,row in promoter_hits.iterrows():
        chrom = row['chrom']
        pos = row['SNP_chr_pos']
        var = row['var_id']
        #for each GxE hit variant, this checks if there is another GxE hit variant
        #within the distance threshold which is not the same variant and is not 
        #at the same position. If so, the row with the variant being checked has
        # a true value for the column 'in_cluster.'
        if promoter_hits[(promoter_hits['chrom']==chrom)&\
                                (abs(promoter_hits['SNP_chr_pos']-pos)<8)&\
                                (promoter_hits['var_id']!=var)&\
                                (promoter_hits['SNP_chr_pos']!=pos)]['var_id'].nunique():
            promoter_hits.loc[i,'in_cluster'] = True
        else:
            promoter_hits.loc[i,'in_cluster'] = False

In [ ]:
prom_clust = promoter_hits.loc[promoter_hits['in_cluster']]['SNP_chr_pos'].unique()   

In [ ]:
clusters = promoter_hits[promoter_hits['in_cluster']]

In [ ]:
#Sorting variants within clusters into named clusters (so more than two variants can be within a cluster
#if they're each within 8 bp of each other)
dist_thresh = 8
cluster_number = 1
sorted_clusters = clusters.sort_values(['chrom','SNP_chr_pos']).reset_index()
for i, row in sorted_clusters.iterrows():
    if i!= len(sorted_clusters)-1:
        if row['chrom']==sorted_clusters.loc[i+1, 'chrom']:
            if abs(row['SNP_chr_pos']-sorted_clusters.loc[i+1, 'SNP_chr_pos'])<dist_thresh:
                sorted_clusters.loc[i, 'clusterID'] = 'Cluster'+str(cluster_number)
                sorted_clusters.loc[i+1,'clusterID'] = 'Cluster'+str(cluster_number)
            else:
                cluster_number+=1
        else:
            cluster_number+=1

In [ ]:
#checking to see if variants within the same clusters share 
#the same significant GxE terms at a relaxed FDR threshold of 0.10.

clusts = sorted_clusters.groupby('clusterID')

for clust, df in clusts:
    shared_gxe_ints = None
    gxe_hits_in_cluster = annotated_gxe[(annotated_gxe['padj']<.1)&(annotated_gxe['var_id'].isin(df['var_id'].unique()))]['gxe'].value_counts()
    shared_gxe_ints = gxe_hits_in_cluster[gxe_hits_in_cluster==len(df)].index.to_list()
    sorted_clusters.loc[df.index, 'Cluster GxE Interactions'] = ','.join(shared_gxe_ints)
    

In [ ]:
#All clusters have at least one shared significant GxE term for all variants in the cluster
#with an FDr threshold of 0.10
sorted_clusters[sorted_clusters['Cluster GxE Interactions']==None]

### Overlapping TFBS Data with Clusters and Promoter Hits

In [ ]:
#importing and pre-processing data from three different yeast transcription factor binding databases

#swiss-regulon
swiss = pd.read_csv('../../swiss_regulon.gff.gz', sep = '\t', header = 1, names = ['chromosome','MOVETO','Site_type','Start','End','Score','Strand','dunno','INFO'])
swiss_good = swiss[swiss['Score']>.95]
swiss_good['TF'] = swiss_good['INFO'].str.split(';').str[2].str.split('=').str[1]
swiss_good['chrom'] = swiss_good['chromosome'].str.split('r').str[1]

#Harbison et. al 2008 from SGD
tfbs_harbison = pd.read_csv('../../harbisonTF.txt.gz', '\t',names=[ "chromosome", "Start", "End",'TF', "Score",'chipEvidence','consSpecies'])
tfbs_harbison= tfbs_harbison[tfbs_harbison['chipEvidence'].isin(['good','weak'])]
tfbs_harbison['chrom'] = tfbs_harbison['chromosome'].str.split('r').str[1]

#ORegAnno
tfbs=pd.read_csv('../../oreganno_tfbs.tsv',sep = '\t')
yeast = tfbs[tfbs['Species']=='Saccharomyces cerevisiae']
yeast['chrom'] = yeast['Chr'].str.split('r').str[1].astype(int).apply(lambda x: roman.toRoman(x))
yeast_tfbs = yeast[yeast['Type']=='TRANSCRIPTION FACTOR BINDING SITE']

In [ ]:
#calculating length of each TFBS
swiss_good['length'] = swiss_good['End']-swiss_good['Start']
tfbs_harbison['length'] = tfbs_harbison['End']-tfbs_harbison['Start']
yeast_tfbs['length'] = yeast_tfbs['End']-yeast_tfbs['Start']

In [ ]:
swiss_good['length'].mean()

In [ ]:
tfbs_harbison['length'].mean()

In [ ]:
yeast_tfbs['length'].mean()

In [ ]:
yeast_tfbs['length'].min()

In [ ]:
yeast_tfbs['length'].max()

In [ ]:
#checking to see if any of the clusters of promoter variants overlap TFBS identified in these three data sets

for i,var in sorted_clusters.iterrows():
    
    in_tfbs = yeast_tfbs[(yeast_tfbs['chrom']==var['chrom'])&(yeast_tfbs['Start']<=var['SNP_chr_pos'])&(yeast_tfbs['End']>=var['SNP_chr_pos'])]
    in_tfbs_harbison = tfbs_harbison_realish[(tfbs_harbison_realish['chrom']==var['chrom'])&(tfbs_harbison_realish['Start']<=var['SNP_chr_pos'])&(tfbs_harbison_realish['End']>=var['SNP_chr_pos'])]
    in_tfbs_swiss = swiss_good[(swiss_good['chrom'] == var['chrom'])&(swiss_good['Start']<=var['SNP_chr_pos'])&(swiss_good['End']>=var['SNP_chr_pos'])]
    if len(in_tfbs_harbison):
        sorted_clusters.loc[i,'inTFBS'] = True
        sorted_clusters.loc[i,'TF'] = in_tfbs_harbison['TF'].values[0]
    elif len(in_tfbs):
        sorted_clusters.loc[i,'inTFBS'] = True
        sorted_clusters.loc[i,'TF'] = in_tfbs['Regulatory_Element_Symbol'].values[0]
    elif len(in_tfbs_swiss):
        sorted_clusters.loc[i,'inTFBS'] = True
        sorted_clusters.loc[i,'TF'] = in_tfbs_swiss['TF'].values[0]
    else:
        sorted_clusters.loc[i,'inTFBS'] = False
        sorted_clusters.loc[i,'TF'] = False
(sorted_clusters.groupby('clusterID')['inTFBS'].agg('sum')>0).sum()

## Plotting figure 3h showing HMG1 promoter

In [ ]:
lov_hits = annotated_fitness[(annotated_fitness['padj']<.01)&(annotated_fitness['condition']=='LOV')&(annotated_fitness['var_id'].isin(gxe_hit_variants))]['var_id'].unique()

In [ ]:

plt.rcParams['font.family']='Arial'
plt.rcParams['font.size'] = 7
fig,ax = plt.subplots(figsize = (7.5,.75))
hmg1_hits = annotated_fitness[(annotated_fitness['Gene']=='HMG1')&(annotated_fitness['condition'].isin(['LOV','CAFF']))&(annotated_fitness['var_id'].isin(lov_hits))&(annotated_fitness['SNP_chr_pos']>118700)]
#grey spans for each cluster
plt.axvspan(119380,119389, alpha = .4, color = 'dimgrey')
plt.axvspan(119366,119372, alpha = .4, color = 'dimgrey')
plt.axvspan(119342,119354, alpha = .4, color = 'dimgrey')
plt.axhline(0, color ='black',linestyle = 'dashed', linewidth =.6)
plt.xlim(118850,119500)
plt.ylim([-.11,.11])

#adding a gene arrow for HMG1
plt.arrow(118898,-0,-48,0, width = .03,head_width = .1, head_length = 0,length_includes_head=True)
plt.annotate('<<HMG1', xy = (118873,-.008), color = 'white', fontstyle = 'oblique', weight = 'bold', ha = 'center')   

plt.ylabel('Fitness')
plt.xlabel('Genomic Coordinates (chr13)')
sns.scatterplot('SNP_chr_pos','coef',hue = 'condition', data = hmg1_hits, s =15, palette = ['chartreuse','purple'])

plt.legend(title = 'Condition')
plt.savefig('../GxE_Figures/Figure_3/fig3f_hmg1_clusters.svg')